In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load the data using pandas

df = pd.read_excel('C:/Users/dayeo/Documents/GitHub/Afo/Finance assignemnt/Coursework_DATA.xls', sheet_name='Sheet1')

In [4]:
df.head()

,id,dscode,Year,LagExcessReturn,CASH,INVENTORIES,CURRENT_ASSETS,PROPERTY_PLANT_EQUIPMENT,TOTAL_INTANGIBLE,TOTAL_ASSETS,...,OPERATING_INCOME,INTEREST_EXPENSE_ON_DEBT,EBIT,PRETAX_INCOME,INCOME_TAXES,NET_INCOME_BEFORE_EXTRA_ITEMS,NET_INCOME_AVAILABLE_TO_COMMON,OPERATING_CASH_FLOW,MARKET_VALUE,EBTDA
0,505,27947W,2013,-0.035987,107505.0,0.0,117515.0,388.0,597.0,119712,...,-12566,0.0,-9639.0,-9639,0,-9639,-9639,-7468,43710.0,-9370.0
1,778,29262P,2013,1.580063,10284.0,0.0,23615.0,72.0,11571.0,35257,...,-1579,77.0,-1043.0,-1120,-174,-946,-946,97,5390.0,-1090.0
2,1037,30463C,2013,-0.894205,5553.0,1.0,8476.0,64.0,7357.0,16427,...,-542,0.0,-497.0,-497,28,-1413,-1413,-323,8510.0,-485.0
3,1400,35607F,2013,-0.373189,1447.0,0.0,1516.0,250.0,3792.0,5558,...,-892,222.0,-2318.0,-5095,0,-5095,-5095,-189,18660.0,NaN
4,4213,999779,2013,0.090696,16072.0,0.0,16505.0,0.0,32531.0,49555,...,-2956,0.0,-1784.0,-1784,0,-1784,-1784,-744,114290.0,-1746.0


In [5]:
df.shape

(11943, 31)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11943 entries, 0 to 11942
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              11943 non-null  int64  
 1   dscode                          11943 non-null  object 
 2   Year                            11943 non-null  int64  
 3   LagExcessReturn                 11943 non-null  float64
 4   CASH                            11943 non-null  float64
 5   INVENTORIES                     11943 non-null  float64
 6   CURRENT_ASSETS                  11943 non-null  float64
 7   PROPERTY_PLANT_EQUIPMENT        11943 non-null  float64
 8   TOTAL_INTANGIBLE                11943 non-null  float64
 9   TOTAL_ASSETS                    11943 non-null  int64  
 10  ACCOUNTS_PAYEABLE               11943 non-null  float64
 11  CURRENT_LIABILITIES             11943 non-null  float64
 12  WORKING_CAPITAL                 

![alt text](image.png)

# Liquidity 
Current assets / Total assets <br>
Cash and equivalent / current liabilities <br>
Current assets / current liabilities <br>
Sales / inventory <br>
Current assets / sales<br>
Current assets / total liabilities<br>

# Solvency 
Total Liabilities / sales<br>
Total liabilities / assets<br>
The book value of equity / Total liabilities<br>
Interest/EBITDA <br>


# Market information 
Lag (excess return) <br>
Market capital <br>
Market value / Total Equity <br>


# Profitability 
Net income / Total assets <br>
Net income / Total Equity <br>

# Asset utilisation 
Working capital / sales <br>
Quick assets / sales <br>

# Cash flow 
Operating cash flow / Total liabilities<br>
Operating cash flow / Total assets<br>

# Making the final ratios - Feature Creation 

In [7]:
# Exploring the columns of the dataframe

df.columns

Index(['id', 'dscode', 'Year', 'LagExcessReturn', 'CASH', 'INVENTORIES',
       'CURRENT_ASSETS', 'PROPERTY_PLANT_EQUIPMENT', 'TOTAL_INTANGIBLE',
       'TOTAL_ASSETS', 'ACCOUNTS_PAYEABLE', 'CURRENT_LIABILITIES',
       'WORKING_CAPITAL', 'TOTAL_DEBT', 'TOTAL_LIABILITIES', 'COMMON_EQUITY',
       'TOTAL_SHAREHOLDERS_EQUITY', 'TOTAL_EQUITY', 'NET_SALES', 'COGSD',
       'GROSS_INCOME', 'OPERATING_INCOME', 'INTEREST_EXPENSE_ON_DEBT', 'EBIT',
       'PRETAX_INCOME', 'INCOME_TAXES', 'NET_INCOME_BEFORE_EXTRA_ITEMS',
       'NET_INCOME_AVAILABLE_TO_COMMON', 'OPERATING_CASH_FLOW', 'MARKET_VALUE',
       'EBTDA'],
      dtype='object')

In [8]:
# Creating financial ratios using lambda functions

# Liquidity (l1) =  Current assets / Total assets 

df['l1'] = df.apply(lambda x: x['CURRENT_ASSETS'] / x['TOTAL_ASSETS'], axis=1)



In [9]:
# Liquidity  (l2) = Cash and equivalent / current liabilities

df['l2'] = df.apply(lambda x: x['CASH'] / x['CURRENT_LIABILITIES'], axis=1)

In [10]:
# Liquidity (l3) = Current assets / current liabilities

df['l3'] = df.apply(lambda x: x['CURRENT_ASSETS'] / x['CURRENT_LIABILITIES'], axis=1)

In [11]:
# Liquidity (l4) = Sales / inventory

# on the first try, I got an error because of division by zero. I had to add a condition to check if the denominator is zero.
df['l4'] = df.apply(lambda x: x['NET_SALES'] / x['INVENTORIES'] if x['INVENTORIES'] != 0 else 0, axis=1)


In [12]:
# Liquidity (l5) = Current assets / sales

df['l5'] = df.apply(lambda x: x['CURRENT_ASSETS'] / x['NET_SALES'] if x['NET_SALES'] != 0 else 0, axis=1)

In [13]:
# Liquidity (l6) = Current liabilities / sales

df['l6'] = df.apply(lambda x: x['CURRENT_LIABILITIES'] / x['NET_SALES'] if x['NET_SALES'] != 0 else 0, axis=1)

In [14]:
# Solvency (s1) = Total Liabilities / sales

df['s1'] = df.apply(lambda x: x['TOTAL_LIABILITIES'] / x['NET_SALES'] if x['NET_SALES'] != 0 else 0, axis=1)

# Solvency (s2) = Total liabilities / assets

df['s2'] = df.apply(lambda x: x['TOTAL_LIABILITIES'] / x['TOTAL_ASSETS'] if x['TOTAL_ASSETS'] != 0 else 0, axis=1)

# Solvency (s3) = The book value of equity / Total liabilities

df['s3'] = df.apply(lambda x: x['TOTAL_SHAREHOLDERS_EQUITY'] / x['TOTAL_LIABILITIES'] if x['TOTAL_LIABILITIES'] != 0 else 0, axis=1)

# Solvency (s4) = Interest/EBITDA

df['s4'] = df.apply(lambda x: x['INTEREST_EXPENSE_ON_DEBT'] / x['EBTDA'] if x['EBTDA'] != 0 else 0, axis=1)

In [15]:
# Market information (m1) = Market value / Total Equity

df['m1'] = df.apply(lambda x: x['MARKET_VALUE'] / x['TOTAL_EQUITY'] if x['TOTAL_EQUITY'] != 0 else 0, axis=1)



In [16]:
# Profitability (p1) = Net income / Total assets

df['p1'] = df.apply(lambda x: x['NET_INCOME_AVAILABLE_TO_COMMON'] / x['TOTAL_ASSETS'] if x['TOTAL_ASSETS'] != 0 else 0, axis=1)

# Profitability (p2) = Net income / Total Equity

df['p2'] = df.apply(lambda x: x['NET_INCOME_AVAILABLE_TO_COMMON'] / x['TOTAL_EQUITY'] if x['TOTAL_EQUITY'] != 0 else 0, axis=1)


In [17]:
# Asset utilisation (a1) = Working capital / sales

df['a1'] = df.apply(lambda x: x['WORKING_CAPITAL'] / x['NET_SALES'] if x['NET_SALES'] != 0 else 0, axis=1)

# Asset utilisation (a2) = Quick assets / sales

df['a2'] = df.apply(lambda x: x['CASH'] / x['NET_SALES'] if x['NET_SALES'] != 0 else 0, axis=1)


In [18]:
# Cash flow (c1) = Operating cash flow / Total liabilities

df['c1'] = df.apply(lambda x: x['OPERATING_CASH_FLOW'] / x['TOTAL_LIABILITIES'] if x['TOTAL_LIABILITIES'] != 0 else 0, axis=1)

# Cash flow (c2) = Operating cash flow / Total assets

df['c2'] = df.apply(lambda x: x['OPERATING_CASH_FLOW'] / x['TOTAL_ASSETS'] if x['TOTAL_ASSETS'] != 0 else 0, axis=1)


In [19]:
df.shape

(11943, 48)

# Cleaning Data
### Excludeing companies that have been listed for less than two years. 


In [20]:
df.columns

Index(['id', 'dscode', 'Year', 'LagExcessReturn', 'CASH', 'INVENTORIES',
       'CURRENT_ASSETS', 'PROPERTY_PLANT_EQUIPMENT', 'TOTAL_INTANGIBLE',
       'TOTAL_ASSETS', 'ACCOUNTS_PAYEABLE', 'CURRENT_LIABILITIES',
       'WORKING_CAPITAL', 'TOTAL_DEBT', 'TOTAL_LIABILITIES', 'COMMON_EQUITY',
       'TOTAL_SHAREHOLDERS_EQUITY', 'TOTAL_EQUITY', 'NET_SALES', 'COGSD',
       'GROSS_INCOME', 'OPERATING_INCOME', 'INTEREST_EXPENSE_ON_DEBT', 'EBIT',
       'PRETAX_INCOME', 'INCOME_TAXES', 'NET_INCOME_BEFORE_EXTRA_ITEMS',
       'NET_INCOME_AVAILABLE_TO_COMMON', 'OPERATING_CASH_FLOW', 'MARKET_VALUE',
       'EBTDA', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 's1', 's2', 's3', 's4',
       'm1', 'p1', 'p2', 'a1', 'a2', 'c1', 'c2'],
      dtype='object')

In [21]:
df.nunique()
# There're 2092 ids in the dataset
df.id.nunique()


2092

In [22]:
df.Year.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], dtype=int64)

In [23]:
# Grouping the data by the id column

df.groupby('id')['Year'].count().reset_index().sort_values('Year')

df_grouped = df.groupby('id')['Year'].count().reset_index()



In [24]:
# Checking for id that have less than 2 years

companies_2 = df_grouped[df_grouped['Year'] < 2]

In [25]:

companies_id_2 = companies_2.id.unique()

companies_2.id.unique()


array([  29,   57,   66,  121,  139,  151,  207,  210,  212,  237,  280,
        287,  328,  339,  344,  348,  351,  358,  363,  372,  421,  429,
        440,  441,  460,  477,  510,  514,  519,  529,  530,  546,  559,
        568,  581,  592,  600,  610,  613,  635,  642,  649,  655,  662,
        672,  673,  696,  703,  716,  756,  769,  771,  779,  797,  800,
        839,  872,  886,  887,  894,  926,  930,  933,  947,  967, 1001,
       1027, 1034, 1068, 1073, 1076, 1103, 1104, 1132, 1162, 1175, 1189,
       1198, 1217, 1269, 1290, 1332, 1462, 1679, 1695, 1867, 1885, 1898,
       1915, 1918, 1926, 1937, 1950, 1956, 2018, 2200, 2205, 2231, 2247,
       2318, 2360, 2449, 2513, 2515, 2518, 2533, 2597, 2651, 2666, 2752,
       2835, 2973, 2996, 3021, 3040, 3059, 3105, 3172, 3247, 3367, 3440,
       3466, 3582, 3667, 3673, 3807, 3836, 3978, 3979, 4084, 4096, 4107,
       4116], dtype=int64)

In [26]:
# removing comanies that have less than 2 years from the dataset

df = df[~df['id'].isin(companies_id_2)]

In [27]:
df.shape

(11810, 48)

In [28]:
# Checking for conmpanies that were removed at random. I checked for company with id 1695. 

df['id'].where(df['id'] == 1695).count()

0

# Removing Missing Values

In [30]:
df.isnull().count()

id                                11810
dscode                            11810
Year                              11810
LagExcessReturn                   11810
CASH                              11810
INVENTORIES                       11810
CURRENT_ASSETS                    11810
PROPERTY_PLANT_EQUIPMENT          11810
TOTAL_INTANGIBLE                  11810
TOTAL_ASSETS                      11810
ACCOUNTS_PAYEABLE                 11810
CURRENT_LIABILITIES               11810
WORKING_CAPITAL                   11810
TOTAL_DEBT                        11810
TOTAL_LIABILITIES                 11810
COMMON_EQUITY                     11810
TOTAL_SHAREHOLDERS_EQUITY         11810
TOTAL_EQUITY                      11810
NET_SALES                         11810
COGSD                             11810
GROSS_INCOME                      11810
OPERATING_INCOME                  11810
INTEREST_EXPENSE_ON_DEBT          11810
EBIT                              11810
PRETAX_INCOME                     11810


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11810 entries, 0 to 11942
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              11810 non-null  int64  
 1   dscode                          11810 non-null  object 
 2   Year                            11810 non-null  int64  
 3   LagExcessReturn                 11810 non-null  float64
 4   CASH                            11810 non-null  float64
 5   INVENTORIES                     11810 non-null  float64
 6   CURRENT_ASSETS                  11810 non-null  float64
 7   PROPERTY_PLANT_EQUIPMENT        11810 non-null  float64
 8   TOTAL_INTANGIBLE                11810 non-null  float64
 9   TOTAL_ASSETS                    11810 non-null  int64  
 10  ACCOUNTS_PAYEABLE               11810 non-null  float64
 11  CURRENT_LIABILITIES             11810 non-null  float64
 12  WORKING_CAPITAL                 11810

# Accounting Variables:

Balance Sheet Items:
CASH, 
INVENTORIES, 
CURRENT_ASSETS, 
PROPERTY_PLANT_EQUIPMENT, 
TOTAL_INTANGIBLE, 
TOTAL_ASSETS, 
ACCOUNTS_PAYABLE, 
CURRENT_LIABILITIES,
WORKING_CAPITAL, 
TOTAL_DEBT, 
TOTAL_LIABILITIES, 
COMMON_EQUITY, 
TOTAL_SHAREHOLDERS_EQUITY,
TOTAL_EQUITY, 
NET_SALES, 
COGS (Cost of Goods Sold), 
GROSS_INCOME, 
OPERATING_INCOME, 
INTEREST_EXPENSE_ON_DEBT, 
EBIT (Earnings Before Interest and Taxes), 
PRETAX_INCOME, 
INCOME_TAXES, 
NET_INCOME_BEFORE_EXTRA_ITEMS,
NET_INCOME_AVAILABLE_TO_COMMON, 
OPERATING_CASH_FLOW. 

# Market-Based Variables:

MARKET_VALUE
EBTDA

In [32]:
df.dropna(inplace=True)

In [33]:
df.shape

(10320, 48)